In [6]:
import numpy as np
from numba import njit
from numpy.random import Generator, MT19937, SeedSequence
sq = SeedSequence(1234)
bit_generator = MT19937(sq)
gen = Generator(bit_generator)
for i in range(10):
    print(gen.random(1))
gen.integers(low=0,high=6,size=1)

[0.12038356]
[0.40370142]
[0.87770263]
[0.9565788]
[0.42646002]
[0.28304326]
[0.90094107]
[0.83083314]
[0.67528993]
[0.3977176]


array([5])

In [7]:
import numpy as np
import numba as nb
from numpy.random import PCG64
from timeit import timeit

bit_gen = PCG64()
next_d = bit_gen.cffi.next_double
state_addr = bit_gen.cffi.state_address

def rando(n, state):
    out = np.empty(n)
    for i in range(n):
        out[i]= next_d(state)
        
    return out

# Compile using Numba
randoj = nb.jit(rando, nopython=True)
# Must use state address not state with numba
n = 10**6

def numbacall():
    return randoj(n, state_addr)

rg = np.random.Generator(PCG64())

def numpycall():
    return rg.random(size=n)

# Check that the functions work
r1 = numbacall()
r2 = numpycall()
assert r1.shape == (n,)
assert r1.shape == r2.shape

t1 = timeit(numbacall, number=1)
print(f'{t1:.2f} secs for {n} PCG64 (Numba/PCG64) gaussian randoms')
t2 = timeit(numpycall, number=1)
print(f'{t2:.2f} secs for {n} PCG64 (NumPy/PCG64) gaussian randoms')


0.01 secs for 1000000 PCG64 (Numba/PCG64) gaussian randoms
0.01 secs for 1000000 PCG64 (NumPy/PCG64) gaussian randoms


In [5]:
import numpy as np
def correl(func,start,dist):
    N = (len(func)-start-dist)
    return 1/N*np.sum(func[start:-dist]*func[start+dist::])-np.sum(func[start:-dist]**2)/N

def autocorrel(func,start,dist):
    c_0 = correl(func,start,0)
    autocorr = 1/2
    cor = correl(func,start,1)/c_0
    iter =1
    while (cor>10**(-4)):
        autocorr += cor
        iter +=1
        cor = correl(func,start,iter)
    return autocorr


In [ ]:
def bootstrap(sample):
    n = len(sample)
    btstrp = np.zeros((n*np.log(n)**2))
    for i in range(n*np.log(n)**2):
        rand = randoj(n,state_addr)
        for pos in range(n):
            btstrp[pos] = sample[int(100*rand[pos])]
    return btstrp

def jackkknife(sample,cut_size,):
    jack_mat = np.zeros((len(sample)-cut_size+1,len(sample)-cut_size))
    for i in range(len(sample)-cut_size+1):
        jack_mat[i,0:i]= sample[0:i]
        jack_mat[i,i::] = sample[i+cut_size::]
    return jack_mat


def jack_mean(sample,cut_size):
    return np.mean(jackkknife(sample,cut_size),axis=1)

def jack_var(sample,cut_size):
    return np.std(jack_mean(sample,cut_size))

def bootstrap_mean(sample):
    mean_mat = np.mean(bootstrap(sample),axis=1)
    return np.std(mean_mat)


